#### Note: 20221212
- sensor會重複寫入無法校正
    - 直接使用

- 篩除掉sensor NaN、mismatch後的資料直接使用


## Filtering potentially high-risk data
- Based on Andy's preprocessed data
    - "data/preprocessed_Andy/..._Target.csv"
    - preprocessed pipeline base on `multi.py`
#### Criteria

- Threshold
    - Speed > 3.0 m/s (10.8 hm/h)
    - gsensor > 0.15g (0.15g = 1.5 m/s^2, 0.3g = 2.94 m/s^2)
- Alignment
    - Ignore timestamps < 10
    - Ignore timestamp mismatches (between gsensor and gyroscope)

#### Time slice cut out from sensor data (sharp changes)
- Establish `event_id` fieled

In [8]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import timedelta
from multiprocessing import Pool
# import feather

dir_data = Path('data')

In [4]:
!ls -l 'data/'
!ls -lh 'data/preprocessed_Andy'

total 1514640
-rwxrwxrwx 1 yunghui yunghui 1550988456 Nov 18 10:45 cars.7z
drwxrwxrwx 1 yunghui yunghui        512 Dec  9 11:13 preprocessed_Andy
drwxrwxrwx 1 yunghui yunghui        512 Dec  8 16:56 preprocessed_CH
total 685M
-rwxrwxrwx 1 yunghui yunghui 6.3M Dec  7 11:35 HR55AD8921_Target.csv
-rwxrwxrwx 1 yunghui yunghui  37M Dec  7 11:35 KA01AF2992_Target.csv
-rwxrwxrwx 1 yunghui yunghui 148M Dec  7 11:35 KA01AF482_Target.csv
-rwxrwxrwx 1 yunghui yunghui  50M Dec  7 11:35 KA04MW6569_Target.csv
-rwxrwxrwx 1 yunghui yunghui 242M Dec  7 11:35 KA51AG8742_Target.csv
-rwxrwxrwx 1 yunghui yunghui  86M Dec  7 11:35 KA51AG8752_Target.csv
-rwxrwxrwx 1 yunghui yunghui 8.2M Dec  7 11:35 MH01DE9816_Target.csv
-rwxrwxrwx 1 yunghui yunghui  33M Dec  7 11:35 MH04JK4981_Target.csv
-rwxrwxrwx 1 yunghui yunghui 380K Dec  7 11:35 MH04JK8738_Target.csv
-rwxrwxrwx 1 yunghui yunghui  77M Dec  7 11:35 MH04KF8187_Target.csv


# Data Loading

In [5]:
filename = 'KA04MW6569'


In [13]:
# df = feather.read_dataframe(f'{filename}.feather')
df = pd.read_csv(dir_data/'preprocessed_Andy'/f'{filename}_Target.csv', index_col=0)
df['timestamp_conv'] = df['timestamp_conv'].astype('datetime64[ms]')
# df = pd.concat(map(pd.read_csv, filename))
df[:10]

,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor,gyro,...,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
14404,14533700,KA04MW6569,13.030617,77.516181,8.261585,104.002220,1,1651160496000,"[{""timeTick"":3934154,""X"":0.34491777420043945,""...","[{""timeTick"":3934154,""X"":-0.0347137451171875,""...",...,0.344918,-0.487084,0.095874,14533700.1,-0.034714,0.005981,-0.011765,False,False,False
14405,14533700,KA04MW6569,13.030617,77.516181,8.261585,104.002220,1,1651160496000,"[{""timeTick"":3934154,""X"":0.34491777420043945,""...","[{""timeTick"":3934154,""X"":-0.0347137451171875,""...",...,0.314513,-0.392850,-0.181710,14533700.2,-0.027267,-0.061874,-0.034241,False,False,False
14406,14533700,KA04MW6569,13.030617,77.516181,8.261585,104.002220,1,1651160496000,"[{""timeTick"":3934154,""X"":0.34491777420043945,""...","[{""timeTick"":3934154,""X"":-0.0347137451171875,""...",...,0.536195,-0.042270,-0.034724,14533700.3,-0.009262,-0.004028,-0.017731,False,False,False
14407,14533700,KA04MW6569,13.030617,77.516181,8.261585,104.002220,1,1651160496000,"[{""timeTick"":3934154,""X"":0.34491777420043945,""...","[{""timeTick"":3934154,""X"":-0.0347137451171875,""...",...,0.044947,-0.010064,-0.224942,14533700.4,-0.024384,-0.020645,-0.018356,False,False,False
14408,14533700,KA04MW6569,13.030617,77.516181,8.261585,104.002220,1,1651160496000,"[{""timeTick"":3934154,""X"":0.34491777420043945,""...","[{""timeTick"":3934154,""X"":-0.0347137451171875,""...",...,0.168777,0.152842,-0.232790,14533700.5,-0.012451,-0.012115,-0.018677,False,False,False
14409,14533704,KA04MW6569,13.030598,77.516253,7.508620,102.957184,1,1651160497000,"[{""timeTick"":3935154,""X"":0.34683752059936523,""...","[{""timeTick"":3935154,""X"":-0.0047760009765625,""...",...,0.346838,0.116714,-0.192143,14533704.1,-0.004776,-0.052170,-0.011871,False,False,False
14410,14533704,KA04MW6569,13.030598,77.516253,7.508620,102.957184,1,1651160497000,"[{""timeTick"":3935154,""X"":0.34683752059936523,""...","[{""timeTick"":3935154,""X"":-0.0047760009765625,""...",...,0.134666,0.173285,-0.045174,14533704.2,0.010986,-0.020004,-0.000153,False,False,False
14411,14533704,KA04MW6569,13.030598,77.516253,7.508620,102.957184,1,1651160497000,"[{""timeTick"":3935154,""X"":0.34683752059936523,""...","[{""timeTick"":3935154,""X"":-0.0047760009765625,""...",...,-0.213028,-0.058266,0.085586,14533704.3,0.000229,-0.010529,0.013809,False,False,False
14412,14533704,KA04MW6569,13.030598,77.516253,7.508620,102.957184,1,1651160497000,"[{""timeTick"":3935154,""X"":0.34683752059936523,""...","[{""timeTick"":3935154,""X"":-0.0047760009765625,""...",...,-0.237432,-0.398034,0.042169,14533704.4,-0.033127,-0.003601,0.000061,False,False,False
14413,14533704,KA04MW6569,13.030598,77.516253,7.508620,102.957184,1,1651160497000,"[{""timeTick"":3935154,""X"":0.34683752059936523,""...","[{""timeTick"":3935154,""X"":-0.0047760009765625,""...",...,0.102970,-0.212659,0.037861,14533704.5,-0.006271,0.029205,-0.011765,False,False,False


In [14]:
print(df.columns)
df.info()

Index(['id', 'vehicle_id', 'latitude', 'longitude', 'speed', 'heading',
       'acc_signal', 'timestamp', 'gsensor', 'gyro', 'timestamp_conv',
       'gsen_X', 'gsen_Y', 'gsen_Z', 'subid', 'gyro_X', 'gyro_Y', 'gyro_Z',
       'unsafe_gsenX', 'unsafe_gsenY', 'unsafe_gsenZ'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 40473 entries, 14404 to 4107668
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              40473 non-null  int64         
 1   vehicle_id      40473 non-null  object        
 2   latitude        40473 non-null  float64       
 3   longitude       40473 non-null  float64       
 4   speed           40473 non-null  float64       
 5   heading         40473 non-null  float64       
 6   acc_signal      40473 non-null  int64         
 7   timestamp       40473 non-null  int64         
 8   gsensor         40473 non-null  object        
 9   gyro       

In [15]:
df.describe()

# Not yet filtered 

,id,latitude,longitude,speed,heading,acc_signal,timestamp,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z
count,4.047300e+04,40473.000000,40473.000000,40473.000000,40473.000000,40473.0,4.047300e+04,40473.000000,40473.000000,40473.000000,4.047300e+04,40473.000000,40473.000000,40473.000000
mean,3.409103e+07,13.049680,77.556480,6.534554,183.202849,1.0,1.657719e+12,0.016572,-0.044124,-0.015594,3.409103e+07,0.000479,-0.001248,0.000556
std,1.218819e+07,0.034538,0.133762,4.096620,120.275444,0.0,3.739235e+09,0.707582,0.951826,0.687863,1.218819e+07,0.074052,0.059970,0.062181
min,1.453370e+07,12.829170,77.398033,0.000602,0.000002,1.0,1.651160e+12,-4.845965,-5.656498,-4.071292,1.453370e+07,-0.611145,-0.634613,-0.501495
25%,2.347323e+07,13.032835,77.508619,4.000906,74.297699,1.0,1.654741e+12,-0.272125,-0.531077,-0.344411,2.347323e+07,-0.016678,-0.018219,-0.017670
50%,3.271389e+07,13.046274,77.512778,5.837009,175.870682,1.0,1.657167e+12,0.000893,0.001315,-0.005700,3.271389e+07,0.000214,-0.000137,0.000000
75%,4.184950e+07,13.061233,77.516388,8.152754,305.065491,1.0,1.659911e+12,0.295524,0.551351,0.287445,4.184950e+07,0.018585,0.017731,0.019379
max,5.697301e+07,13.198758,78.010177,28.438448,359.981323,1.0,1.664799e+12,5.266725,4.348207,6.423663,5.697301e+07,0.622650,0.408310,0.548920


In [175]:
df.iloc[20:30,:9]

,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor
14424,14533718,KA04MW6569,13.030544,77.516465,6.214806,103.905037,1,1651160500000,"[{""timeTick"":3938155,""X"":0.1266622543334961,""Y..."
14425,14533718,KA04MW6569,13.030544,77.516465,6.214806,103.905037,1,1651160500000,"[{""timeTick"":3938155,""X"":0.1266622543334961,""Y..."
14426,14533718,KA04MW6569,13.030544,77.516465,6.214806,103.905037,1,1651160500000,"[{""timeTick"":3938155,""X"":0.1266622543334961,""Y..."
14427,14533718,KA04MW6569,13.030544,77.516465,6.214806,103.905037,1,1651160500000,"[{""timeTick"":3938155,""X"":0.1266622543334961,""Y..."
14428,14533718,KA04MW6569,13.030544,77.516465,6.214806,103.905037,1,1651160500000,"[{""timeTick"":3938155,""X"":0.1266622543334961,""Y..."
14429,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y..."
14430,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y..."
14431,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y..."
14432,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y..."
14433,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y..."


In [176]:
df.iloc[20:30, 9+2:]

,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
14424,0.126662,-1.055346,0.823699,14533718.1,-0.005096,0.001938,0.024567,False,False,False
14425,-0.265245,-1.468928,0.753511,14533718.2,-0.010849,-0.004883,0.018173,False,False,False
14426,0.676355,-0.574580,1.203745,14533718.3,0.055298,-0.015213,-0.018890,False,False,False
14427,-1.250422,-2.332044,-0.076195,14533718.4,-0.141571,0.090775,0.115219,False,False,False
14428,-1.301949,-1.207870,-0.529979,14533718.5,-0.213791,-0.028214,0.056946,False,False,False
14429,3.183076,2.567991,2.252641,14533728.1,0.176208,0.050629,-0.088562,True,False,False
14430,-0.794319,-0.029197,-1.163878,14533728.2,0.209335,-0.133133,-0.159302,False,False,False
14431,-0.070409,-0.024428,-0.600643,14533728.3,-0.019272,-0.126419,0.010406,False,False,False
14432,2.555487,2.778320,1.741617,14533728.4,0.152771,0.131577,-0.121689,False,False,False
14433,-4.042803,-3.189753,-3.758142,14533728.5,-0.095108,-0.197800,0.108398,True,True,True


In [169]:
df.filter(regex='unsafe')[20:30]

,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
20,False,False,False
21,False,False,False
22,False,False,False
23,False,False,False
24,False,False,False
25,True,False,False
26,False,False,False
27,False,False,False
28,False,False,False
29,True,True,True


In [150]:
# feather.write_dataframe(df, f'{filename}.feather')


In [151]:
# col_sel = ['id', 'vehicle_id', 'latitude', 'longitude', 'speed', 'heading', 'acc_signal', 'timestamp', 'timestamp_conv']
# col_sel += ['gsensor', 'gyro', 'gsen_X', 'gsen_Y', 'gsen_Z', 'gyro_X', 'gyro_Y', 'gyro_Z', 'subid']
# col_sel
# df = df[col_sel]
# df

## Split json

In [152]:
# str to list. [dict()]。 
# [{'timeTick': v, 'X': v, 'Y': v, 'Z': v}]
# df['gsensor_json'] = [json.loads(x) for x in df.gsensor]
# df['gyro_json'] = [json.loads(x) for x in df.gyro]      

# print(type(df.gsensor[0]), ' to ', type(df['gsensor_json'][0]))
# df['gsensor_json'][0][0]


In [153]:
# def split_json(df: pd.DataFrame) -> pd.DataFrame:
#     for item in ['timeTick', 'X', 'Y', 'Z']:
#         df[f'gsen_{item}'] = [[x[f'{item}'] for x in xs] for xs in df['gsensor_json']]
#         df[f'gyro_{item}'] = [[x[f'{item}'] for x in xs] for xs in df['gyro_json']]
        
#     return df


In [154]:
# split_json(df) 
# df.drop(columns=['acc_signal', 'gyro', 'gsensor', 'gsensor_json', 'gyro_json'], inplace=True)
# df.head()


## file saving

In [155]:
# feather.write_dataframe(df, f'df_preProcess_{filename}.feather')


# Filtering

In [156]:
# # convert 13 digits unix ts to date time
# df.loc[:,'timestamp_conv'] = pd.to_datetime(df['timestamp'], unit='ms')

# # filter out 車子熄火的狀態 
# # df = df[df['acc_signal'] == 1]

# # filter out low speed (5.5 m/s = 20 kph; 2.7 m/s = 10kph)  
# speed = 2.7  # threshold of Andy : 3.0

# df = df[df['speed'] > speed]

# df.reset_index(inplace=True, drop=True)
# # filter out low Z-axis gsensor (0.15g = 1.5 m/s^2, 0.3g = 2.94 m/s^2)
# threshold = 2.94

# df['filter_gsen_X'] = [True if (np.max(x, initial=0) > threshold) | (np.min(x, initial=0) < -threshold) else False for x in df.gsen_X] 
# df['filter_gsen_Y'] = [True if (np.max(x, initial=0) > threshold) | (np.min(x, initial=0) < -threshold) else False for x in df.gsen_Y] 
# df['filter_gsen_Z'] = [True if (np.max(x, initial=0) > threshold) | (np.min(x, initial=0) < -threshold) else False for x in df.gsen_Z] 

# df_outlier = df[(df['filter_gsen_X'] == True) | (df['filter_gsen_Y'] == True) | (df['filter_gsen_Z'] == True)]

#### Criteria
- Threshold
    - Speed > 3.0 m/s (10.8 hm/h)
    - gsensor > 0.15g (0.15g = 1.5 m/s^2, 0.3g = 2.94 m/s^2)

- 以秒為單位
    - 需要將1秒內的5筆sensor採樣點資料一起打包

- 以timestamp為單位
    - 或直接以異常值發生的timestamp當單位，往前後採用
    
- 同樣都要處理gps 採樣與 sensor採樣不一致的問題

- sensor data 問題
    - 重複寫入
    - 沒有對齊
        - gyro與gsensor寫入不同步
    - 缺失(gyro與gsensor同步缺失)

In [ ]:
df.iloc[20:30, 9+2:]

- 這個篩選是以timestamp為單位，無法保留一秒內的完整採樣資料
- 需要改以＂秒＂為單位，保留整批次採樣資料

In [36]:
con_gsen = (df['unsafe_gsenX'] == True) | (df['unsafe_gsenY'] == True) | (df['unsafe_gsenZ'] == True)
con_speed = df['speed'] > 3.0

In [37]:
df[con_gsen & con_speed]

,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor,gyro,...,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
14429,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y...","[{""timeTick"":3939155,""X"":0.17620849609375,""Y"":...",...,3.183076,2.567991,2.252641,14533728.1,0.176208,0.050629,-0.088562,True,False,False
14433,14533728,KA04MW6569,13.030531,77.516514,5.504738,101.953148,1,1651160501000,"[{""timeTick"":3939157,""X"":3.1830759048461914,""Y...","[{""timeTick"":3939155,""X"":0.17620849609375,""Y"":...",...,-4.042803,-3.189753,-3.758142,14533728.5,-0.095108,-0.197800,0.108398,True,True,True
21044,14539046,KA04MW6569,12.935314,77.536465,3.626204,118.965805,1,1651161833000,"[{""timeTick"":5270040,""X"":0.2576866149902344,""Y...","[{""timeTick"":5270040,""X"":-0.0821380615234375,""...",...,-2.761128,-3.288322,-0.578127,14539046.2,-0.197937,0.073746,0.119522,False,True,False
26476,16600513,KA04MW6569,12.933928,77.538801,4.593207,299.664276,1,1651634053000,"[{""timeTick"":477491175,""X"":1.372842788696289,""...","[{""timeTick"":477491175,""X"":0.1339569091796875,...",...,-3.069280,-2.316227,-1.660519,16600513.3,-0.105515,0.030426,0.054260,True,False,False
26595,16600585,KA04MW6569,12.934813,77.537266,3.544833,300.971924,1,1651634077000,"[{""timeTick"":477515181,""X"":0.7608447074890137,...","[{""timeTick"":477515180,""X"":0.07452392578125,""Y...",...,-2.964461,-0.817166,-2.036281,16600585.2,-0.028061,0.057266,0.025177,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095232,56931429,KA04MW6569,13.032693,77.520603,7.673488,12.993739,1,1664789540434,"[{""timeTick"":129735195,""X"":1.42252779006958,""Y...","[{""timeTick"":129735194,""X"":-0.0021820068359375...",...,1.840670,3.633478,-0.763492,56931429.4,0.026047,-0.054764,-0.142960,False,True,False
4095233,56931429,KA04MW6569,13.032693,77.520603,7.673488,12.993739,1,1664789540434,"[{""timeTick"":129735195,""X"":1.42252779006958,""Y...","[{""timeTick"":129735194,""X"":-0.0021820068359375...",...,-0.137802,-3.533251,0.745377,56931429.5,-0.015503,0.065186,0.168625,False,True,False
4095246,56931442,KA04MW6569,13.032919,77.520658,6.090010,12.693027,1,1664789544455,"[{""timeTick"":129739197,""X"":0.6400856971740723,...","[{""timeTick"":129739197,""X"":0.00091552734375,""Y...",...,0.955173,2.954717,-1.012249,56931442.3,0.013153,-0.066589,-0.179504,False,True,False
4105112,56970605,KA04MW6569,13.047241,77.508049,7.165850,181.923080,1,1664798704812,"[{""timeTick"":138899071,""X"":-0.0773973464965820...","[{""timeTick"":138899071,""X"":-0.036102294921875,...",...,0.646813,2.951618,-0.789567,56970605.2,0.043365,-0.018524,-0.246399,False,True,False


##### Check number of data per timestamp 

In [42]:
df.groupby('timestamp').id.count().sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  15,
        18,  20,  25,  30,  40,  45,  50,  60,  65,  70,  75,  93,  99,
       105, 120, 135, 165, 195, 207, 219, 225, 240, 255, 267])

In [33]:
df.groupby('id').id.count().sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 15, 18, 20])

In [54]:
mask = df.groupby('id').id.count() == 1
idx_ = df.groupby('id').id.count()[mask].index

id = idx_[0]
df.query('id==@id')

,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor,gyro,...,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
686894,20090421,KA04MW6569,13.031567,77.512923,4.041155,9.479123,1,1653268144000,"[{""timeTick"":2111582139,""X"":-0.747638702392578...","[{""timeTick"":2111582139,""X"":-0.070663452148437...",...,-0.747639,0.964187,-0.009824,20090421.1,-0.070663,-0.066895,-0.217941,False,False,False


In [49]:
ts_c = "2022-05-23 01:09:04"
df.query('timestamp_conv==@ts_c').gsensor.values

array(['[{"timeTick":2111582139,"X":-0.7476387023925781,"Y":0.9641866683959961,"Z":-0.009823799133300781}]'],
      dtype=object)

In [53]:
mask = df.groupby('id').id.count() == 20
idx_ = df.groupby('id').id.count()[mask].index


id = idx_[0]
print(ts_c, len(df.query('id==@id').gsensor.values))
df.query('id==@id')

20090421 20


,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor,gyro,...,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
2527581,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.164557,-0.301471,0.123606,36552501.1,0.009918,-0.000793,-0.009598,False,False,False
2527582,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,-0.000784,-0.221695,0.134298,36552501.2,0.013535,0.001862,-0.000229,False,False,False
2527583,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.147655,-0.139551,0.141956,36552501.3,0.024506,-0.007614,-0.006302,False,False,False
2527584,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.096092,-0.102794,0.083211,36552501.4,0.046234,0.004105,-0.013000,False,False,False
2527585,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.114291,1.021729,0.693035,36552501.5,0.099609,0.004318,-0.014603,False,False,False
2527581,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.164557,-0.301471,0.123606,36552501.1,0.009918,-0.000793,-0.009598,False,False,False
2527582,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,-0.000784,-0.221695,0.134298,36552501.2,0.013535,0.001862,-0.000229,False,False,False
2527583,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.147655,-0.139551,0.141956,36552501.3,0.024506,-0.007614,-0.006302,False,False,False
2527584,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.096092,-0.102794,0.083211,36552501.4,0.046234,0.004105,-0.013000,False,False,False
2527585,36552501,KA04MW6569,13.042435,77.514638,8.346089,111.50071,1,1657887300000,"[{""timeTick"":1293552183,""X"":0.1645574569702148...","[{""timeTick"":1293552183,""X"":0.009918212890625,...",...,0.114291,1.021729,0.693035,36552501.5,0.099609,0.004318,-0.014603,False,False,False



- 用 "id"  做groupbyzp分組計算 sensor data資料數量，仍有 1 - 20 筆的變化
    - 實際統計結果 
        - [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 15, 18, 20]   
    - multi.py 
        - 已刪除過<10的資料
        - gsensor與gyro資料數量已經對齊過

In [ ]:
    # data = data[data['speed'] > 3]
    # data = data[
    #     (data['unsafe_gsenX'] == True) | (data['unsafe_gsenY'] == True) | (data['unsafe_gsenZ'] == True)]

## file saving

In [159]:
# feather.write_dataframe(df_outlier.reset_index(drop=True), f'df_outlier_{filename}.feather')


# Zooming

In [160]:
# items = df_outlier.index

# pic_n =1
# df_event = pd.DataFrame()

# for item in items:
#     df_zoom = df[df.index == item].copy()
# #     start = item - 1 # -1 index
# #     end = item + 1 # +1 index
# #     df_zoom = df[(df.index >= start) & (df.index <= end)].copy()
    
# #     # filtering critiria 
# #     if (int(df[df.index == item].timestamp - int(df[df.index == start].timestamp)) <= 950) | \
# #     (int(df[df.index == item].timestamp) - int(df[df.index == start].timestamp) >= 1000) | \
# #     (int(df[df.index == end].timestamp) - int(df[df.index == item].timestamp) <= 950) | \
# #     (int(df[df.index == end].timestamp) - int(df[df.index == item].timestamp) >= 1000) :
# #         continue
    
#     # filter OUT more than 3 or less than 2 records within 3s
#     # if (df_zoom.iloc[:,1].count() > 3) or (df_zoom.iloc[:,1].count() <= 1) :
#     #     continue
#     # # time range between the last and the first records must longer than 3s    
#     # if (df_zoom['timestamp'].iloc[-1] - df_zoom['timestamp'].iloc[0]) < 3000 :
#     #     continue
        
#     df_zoom.loc[:,'event_id'] = pic_n
#     df_event = pd.concat([df_event, df_zoom], axis=0)
    
#     pic_n = pic_n + 1

In [6]:
df.timestamp_conv

14404          2022-04-28 15:41:36
14405          2022-04-28 15:41:36
14406          2022-04-28 15:41:36
14407          2022-04-28 15:41:36
14408          2022-04-28 15:41:36
                    ...           
4107664    2022-10-03 12:17:28.965
4107665    2022-10-03 12:17:28.965
4107666    2022-10-03 12:17:28.965
4107667    2022-10-03 12:17:28.965
4107668    2022-10-03 12:17:28.965
Name: timestamp_conv, Length: 40473, dtype: object

In [ ]:
#  items = data['timestamp_conv'].unique()

def threading_item2(target):
    pic_n = target[0]
    item = target[1]
    dataframe = target[2]
    draw_list = []
    start = item - np.timedelta64(5, 's')  # -5 sesonds
    end = item + np.timedelta64(5, 's')  # +5 seconds
    df_zoom = dataframe[(dataframe['timestamp_conv'] >= start) & (dataframe['timestamp_conv'] <= end)]
    if df_zoom['timestamp_conv'].nunique() < 7:
        return
    # time range between the last and the first records must longer than 3s
    if df_zoom['timestamp_conv'].iloc[-1] - df_zoom['timestamp_conv'].iloc[0] < timedelta(seconds=3):
        return
    df_dict = df_zoom.to_dict()
    draw_list.append([pic_n, df_dict, 0])
    return draw_list


In [19]:
items = df['timestamp_conv'].unique()
print(len(items))


6446


[]

In [32]:

t_slice = 1

draw_list = []

for pic_n, t_stamp_ in enumerate(items):
    start = t_stamp_ - np.timedelta64(t_slice, 's')  # - t_slice sesonds
    end = t_stamp_ + np.timedelta64(t_slice, 's')  # + t_slice seconds
    df_zoom = df[(df['timestamp_conv'] >= start) & (df['timestamp_conv'] <= end)]
    # if df_zoom['timestamp_conv'].nunique() < 1:
    #     continue
    # # time range between the last and the first records must longer than 3s
    # if df_zoom['timestamp_conv'].iloc[-1] - df_zoom['timestamp_conv'].iloc[0] < timedelta(seconds=3):
    #     continue
    df_dict = df_zoom.to_dict()
    draw_list.append([pic_n, df_dict, 0])
    
    if pic_n==10:
        break
    
len(draw_list)


12

In [35]:
draw_list[-1][1]

{'id': {21018: 14539026,
  21019: 14539026,
  21020: 14539026,
  21021: 14539026,
  21022: 14539026,
  21023: 14539030,
  21024: 14539030,
  21025: 14539030,
  21026: 14539030,
  21027: 14539030},
 'vehicle_id': {21018: 'KA04MW6569',
  21019: 'KA04MW6569',
  21020: 'KA04MW6569',
  21021: 'KA04MW6569',
  21022: 'KA04MW6569',
  21023: 'KA04MW6569',
  21024: 'KA04MW6569',
  21025: 'KA04MW6569',
  21026: 'KA04MW6569',
  21027: 'KA04MW6569'},
 'latitude': {21018: 12.935417,
  21019: 12.935417,
  21020: 12.935417,
  21021: 12.935417,
  21022: 12.935417,
  21023: 12.9353945,
  21024: 12.9353945,
  21025: 12.9353945,
  21026: 12.9353945,
  21027: 12.9353945},
 'longitude': {21018: 77.5362628,
  21019: 77.5362628,
  21020: 77.5362628,
  21021: 77.5362628,
  21022: 77.5362628,
  21023: 77.5363113,
  21024: 77.5363113,
  21025: 77.5363113,
  21026: 77.5363113,
  21027: 77.5363113},
 'speed': {21018: 5.12672233581543,
  21019: 5.12672233581543,
  21020: 5.12672233581543,
  21021: 5.12672233581543,

In [33]:
df_zoom

,id,vehicle_id,latitude,longitude,speed,heading,acc_signal,timestamp,gsensor,gyro,...,gsen_X,gsen_Y,gsen_Z,subid,gyro_X,gyro_Y,gyro_Z,unsafe_gsenX,unsafe_gsenY,unsafe_gsenZ
21018,14539026,KA04MW6569,12.935417,77.536263,5.126722,117.089844,1,1651161828000,"[{""timeTick"":5265039,""X"":-0.0822439193725586,""...","[{""timeTick"":5265039,""X"":-0.00714111328125,""Y""...",...,-0.082244,1.450797,-0.881469,14539026.1,-0.007141,-0.001663,0.008194,False,False,False
21019,14539026,KA04MW6569,12.935417,77.536263,5.126722,117.089844,1,1651161828000,"[{""timeTick"":5265039,""X"":-0.0822439193725586,""...","[{""timeTick"":5265039,""X"":-0.00714111328125,""Y""...",...,0.575592,2.322385,-0.977712,14539026.2,0.110672,-0.081558,-0.086090,False,False,False
21020,14539026,KA04MW6569,12.935417,77.536263,5.126722,117.089844,1,1651161828000,"[{""timeTick"":5265039,""X"":-0.0822439193725586,""...","[{""timeTick"":5265039,""X"":-0.00714111328125,""Y""...",...,0.589401,2.856160,-0.956428,14539026.3,0.150421,-0.019455,-0.138718,False,False,False
21021,14539026,KA04MW6569,12.935417,77.536263,5.126722,117.089844,1,1651161828000,"[{""timeTick"":5265039,""X"":-0.0822439193725586,""...","[{""timeTick"":5265039,""X"":-0.00714111328125,""Y""...",...,-2.551582,-0.322374,-2.340988,14539026.4,-0.056351,0.004303,0.047821,False,False,False
21022,14539026,KA04MW6569,12.935417,77.536263,5.126722,117.089844,1,1651161828000,"[{""timeTick"":5265039,""X"":-0.0822439193725586,""...","[{""timeTick"":5265039,""X"":-0.00714111328125,""Y""...",...,-0.483431,0.316703,-1.203639,14539026.5,-0.122406,0.056610,0.138260,False,False,False
21023,14539030,KA04MW6569,12.935394,77.536311,5.110618,116.019669,1,1651161829000,"[{""timeTick"":5266039,""X"":1.094522476196289,""Y""...","[{""timeTick"":5266038,""X"":-0.1475372314453125,""...",...,1.094522,1.821172,0.446883,14539030.1,-0.147537,0.132446,0.101395,False,False,False
21024,14539030,KA04MW6569,12.935394,77.536311,5.110618,116.019669,1,1651161829000,"[{""timeTick"":5266039,""X"":1.094522476196289,""Y""...","[{""timeTick"":5266038,""X"":-0.1475372314453125,""...",...,-1.137633,1.094995,-1.387941,14539030.2,0.140076,-0.073563,-0.121552,False,False,False
21025,14539030,KA04MW6569,12.935394,77.536311,5.110618,116.019669,1,1651161829000,"[{""timeTick"":5266039,""X"":1.094522476196289,""Y""...","[{""timeTick"":5266038,""X"":-0.1475372314453125,""...",...,0.046257,0.371516,-0.989612,14539030.3,-0.025467,-0.008591,0.053894,False,False,False
21026,14539030,KA04MW6569,12.935394,77.536311,5.110618,116.019669,1,1651161829000,"[{""timeTick"":5266039,""X"":1.094522476196289,""Y""...","[{""timeTick"":5266038,""X"":-0.1475372314453125,""...",...,0.012920,0.728843,0.046396,14539030.4,-0.036331,0.054153,0.010422,False,False,False
21027,14539030,KA04MW6569,12.935394,77.536311,5.110618,116.019669,1,1651161829000,"[{""timeTick"":5266039,""X"":1.094522476196289,""Y""...","[{""timeTick"":5266038,""X"":-0.1475372314453125,""...",...,-0.485529,0.906054,-0.665036,14539030.5,0.053040,-0.003052,-0.047943,False,False,False


In [161]:
items = df_outlier['timestamp'].unique()

pic_n = 1
df_event = pd.DataFrame()

for item in items:
    start = item - 1000 # -1 sesonds
    end = item + 1000 # +1 seconds
    df_zoom = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)].copy()
    
    # filtering critiria 
    # filter OUT more than 3 or less than 2 records within 3s
    if (df_zoom.iloc[:,1].count() > 3) or (df_zoom.iloc[:,1].count() <= 1) :
        continue
    # # time range between the last and the first records must longer than 3s    
    # if (df_zoom['timestamp'].iloc[-1] - df_zoom['timestamp'].iloc[0]) < 3000 :
    #     continue
        
    df_zoom.loc[:,'event_id'] = pic_n
    df_event = pd.concat([df_event, df_zoom], axis=0)
    
    pic_n = pic_n + 1

In [162]:

df_event

""


In [ ]:
len(df_event)

2116

## file saving

In [ ]:
# feather.write_dataframe(df_event.reset_index(drop=True), f'df_event_{filename}.feather')


# Stats making

In [ ]:

df_event_agg = df_event.groupby(by='event_id').agg({'gsen_X': 'sum', 'gsen_Y': 'sum', 'gsen_Z': 'sum', \
                                    'gyro_X': 'sum', 'gyro_Y': 'sum', 'gyro_Z': 'sum', \
                                    'latitude': 'mean', \
                                    'longitude': 'mean', \
                                    'speed': ['mean', 'std'], \
                                    'vehicle_id': 'first'})

df_event_agg.columns = ['gsen_X', 'gsen_Y', 'gsen_Z', \
                        'gyro_X', 'gyro_Y', 'gyro_Z', \
                        'lat_mean', 'long_mean', \
                        'speed_mean', 'speed_std', 'vehicle_id']


In [ ]:
def make_stats(df: pd.DataFrame) -> pd.DataFrame:
    for item in ['X', 'Y', 'Z']:
        df[f'gsen_{item}_mean'] = [np.mean(x) if len(x) > 0 else 0 for x in df[f'gsen_{item}']]
        df[f'gsen_{item}_std'] = [np.std(x) if len(x) > 0 else 0 for x in df[f'gsen_{item}']]
        df[f'gsen_{item}_max'] = [np.max(x) if len(x) > 0 else 0 for x in df[f'gsen_{item}']]
        df[f'gsen_{item}_min'] = [np.min(x) if len(x) > 0 else 0 for x in df[f'gsen_{item}']]
        df[f'gsen_{item}_range'] = [np.ptp(x) if len(x) > 0 else 0 for x in df[f'gsen_{item}']]
        df[f'gyro_{item}_mean'] = [np.mean(x) if len(x) > 0 else 0 for x in df[f'gyro_{item}']]
        df[f'gyro_{item}_std'] = [np.std(x) if len(x) > 0 else 0 for x in df[f'gyro_{item}']]
        df[f'gyro_{item}_max'] = [np.max(x) if len(x) > 0 else 0 for x in df[f'gyro_{item}']]
        df[f'gyro_{item}_min'] = [np.min(x) if len(x) > 0 else 0 for x in df[f'gyro_{item}']]
        df[f'gyro_{item}_range'] = [np.ptp(x) if len(x) > 0 else 0 for x in df[f'gyro_{item}']]
     
    return df

In [ ]:
df_event_agg = make_stats(df_event_agg)



In [ ]:
len(df_event_agg)



798

## file saving

In [ ]:
feather.write_dataframe(df_event_agg.reset_index(), f'df_event_agg_{filename}.feather')
